**參考網站：**

***https://www.tensorflow.org/tutorials/images/cnn***

***https://www.tensorflow.org/tutorials/keras/save_and_load***


In [ ]:
import tensorflow as tf

import os
from tensorflow.keras import datasets, layers, models ,backend, callbacks
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
from IPython.display import display as ds
import time
import numpy as np
import tempfile
import pathlib
import requests
import shutil
from lxml import etree

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
path =  os.getcwd() 

if(not os.path.exists(path+r"\images")):
   os.mkdir("images")

path=path.replace("\\", "/")

path=path+"/images/"

print( path )

Start Cifar100 CNN Model Test

In [ ]:
cifar100_labels = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
backend.clear_session()

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(512, (1, 1), activation='relu'))


model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(100))
#model.summary()

In [ ]:
es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels,batch_size=700,
                    epochs=50, 
                    validation_data=(test_images, test_labels),
                   callbacks=[es])

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
history.history

In [ ]:
print(test_acc)

In [ ]:
test_images.shape

In [ ]:
def img_fromArray(imgArr,view=True):  
  imgFromArr=Image.fromarray(np.uint8(imgArr*255))
  if(view):
    ds(imgFromArr)
  return imgFromArr   

In [ ]:
def imgTensorNp(imgnp):
    return tf.image.convert_image_dtype(np.array([imgnp]), tf.float32)

In [ ]:
resu , resu_real , resu_real_sort , real_labelval , real_labelval_sort = 0,0,0,0,0

In [ ]:
def imgPredict(TestImgNp,myModel,have_origin_img_np=False,origin_img_np=None):
  global resu , resu_real , resu_real_sort , real_labelval , real_labelval_sort
  if(have_origin_img_np):
        img_fromArray(origin_img_np,True)
  else:
    img_fromArray(TestImgNp,True)
  tensorImg=imgTensorNp(TestImgNp)
  resu=myModel.predict(tensorImg)
  resu_real=np.array([])
  real_labelval=np.array([])

  for i in range(0,len(resu[0])):
    resui=resu[0,i]

    predictValue=-100

    if(resui>predictValue):
      resu_real=np.append(resu_real,resu[0,i])
      real_labelval=np.append(real_labelval,i)
  resu_real_sort=np.sort(resu_real)
  real_labelval_sort=np.zeros(len(real_labelval),dtype=int)
  real_labelval=real_labelval.astype('int') 

  for i in range(0,len(resu_real)):
    for ii in range(0,len(resu_real_sort)):
      if(resu_real_sort[ii]==resu_real[i]):
        real_labelval_sort[ii]=real_labelval[i]
  
  resu_real_sort=resu_real_sort[::-1]
  real_labelval_sort=real_labelval_sort[::-1]

  lenght=3

  for i in range(0,len(resu_real_sort[0:(lenght)])): 

    print((i+1),
          "\n\n品種:",cifar100_labels[real_labelval_sort[i]],
          "\n準確率:",resu_real_sort[i],
          "\n編號",real_labelval_sort[i],
          "\n"
          )
    

In [ ]:
model_path=path+"my_model"

In [ ]:
model.save(model_path)

In [ ]:
new_model = tf.keras.models.load_model(model_path)

In [ ]:
dir(new_model)

In [ ]:
def download_and_resize_image(url, new_width=32, new_height=32,display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  #response = urlopen(url)
  #image_data = response.read()
  #image_data = BytesIO(image_data)
  image_data = url
  new_width1=int(new_width) 
  new_height1=int(new_height) 
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image,(32,32), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=100)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename

In [ ]:
imgNum=650
imgPredict(test_images[imgNum],new_model)
print("Real Value : "+cifar100_labels[test_labels[imgNum,0]]+" at "+str(test_labels[imgNum,0]))

In [ ]:
path

In [ ]:
my_imgs_now=[]
dirPath = os.walk(path)
img_dir_path=list(dirPath)[0][2]
for i in img_dir_path:
    if(i.split('.')[-1]=="jpg"):
        my_imgs_now.append(i)
#list(enumerate(dirPath))[0]

In [ ]:
def get_correct_url(base_url,part_url):
  if(part_url.split('/')[0].find('http')==-1):
    if(part_url.split('/')[0]==''):
      if(part_url.split('/')[0]==''):
        if(part_url.split('/')[1]==''):
          if(part_url.split('/')[2]==base_url.split('/')[2]):
            return "http:"+part_url
          else:
            pass  
        else:
          if(part_url.split('/')[1]==base_url.split('/')[2]):
            return "http:/"+part_url
          else:
            return "http://"+base_url.split("/")[2]+part_url 
      else:  
        return "http://"+base_url.split("/")[2]+"/"+part_url 
    else:
      return "http://"+base_url.split("/")[2]+"/"+part_url       

  else:
    return part_url


def png_to_jpg(my_img_name):
  im = Image.open(path+my_img_name)
  rgb_im = im.convert('RGB')
  rgb_im.save(path+my_img_name+".jpg")

def my_save_img():
  global my_imgs_now
  def my_save_url_img(my_img_url): 
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}
    r = requests.get(my_img_url,headers=headers, stream=True)
    if r.status_code == 200:
        with open(str((path+my_img_url.split('/')[-1].split('?')[0])), 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)

  url_01=input("請輸入圖片網址：")
  url_01_type=url_01.split('/')[-1].split('.')[-1].split('?')[0]

  
  if(url_01_type=='jpg'):
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0])
  
  elif(url_01_type=='png'):
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0]+".jpg")
    png_to_jpg(url_01.split('/')[-1].split('?')[0])

  else:
    try:
      #response.status_code,response.content,response.text
      headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}
      my_response = requests.get(url_01,headers=headers)
      my_content = my_response.content.decode()
      my_html = etree.HTML(my_content)
      my_imgs = my_html.xpath('.//img/@src')
      print(my_imgs,"\n")
      for i in my_imgs:
        try:
          if(i.split('/')[-1].split('.')[-1].split('?')[0]=='png'):
            #change this line
            my_save_url_img(get_correct_url(url_01,i))
            my_imgs_now.append(i.split('/')[-1].split('?')[0]+".jpg")
            png_to_jpg(i.split('/')[-1].split('?')[0])
          elif(i.split('/')[-1].split('.')[-1].split('?')[0]=='jpg'):
            #change this line
            my_save_url_img(get_correct_url(url_01,i))
            my_imgs_now.append(i.split('/')[-1].split('?')[0])    
        except:
          pass
    except:
      pass  


  if(len(my_imgs_now)==0 or url_01==''):
    print('圖片網址設定失敗，使用預設圖片')
    url_01="https://64.media.tumblr.com/464a8a1770ac6b2c09c80232911673b7/b6713c87a0ba23de-b6/s1280x1920/0306a7f325e1c10e768a26ac2e1ea805ad3211d5.jpg"
    url_01="https://www.collinsdictionary.com/images/full/apple_158989157.jpg"
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0])  
  else:  
    print('圖片網址設定成功')

def get_image_size(my_img):
  im = Image.open(my_img)
  return im.size

In [ ]:
my_save_img()

remove_img=list(enumerate(my_imgs_now[0:]))
for i in remove_img:
    img_repeat=i[1]
    for ii in remove_img[i[0]+1:]:
        if(img_repeat==ii[1]):
            my_imgs_now.remove(img_repeat) 
            break
            
print(my_imgs_now) 

my_imgs_now_np=[]
my_imgs_now_origin_np=[]
for i in range(0,len(my_imgs_now)):
  image = Image.open(download_and_resize_image(path+my_imgs_now[i]))   # image is a PIL image 
  my_imgs_now_np.append( np.array(image)/255 )
  image = Image.open(path+my_imgs_now[i])
  my_imgs_now_origin_np.append( np.array(image)/255 )

In [ ]:
my_imgs_now_np[0].shape

In [ ]:
for i in range(0,len(my_imgs_now_np)):
    imgPredict(my_imgs_now_np[i],
               new_model,
               True,
               my_imgs_now_origin_np[i]
              )